In [14]:
import torch
import torch.cuda
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [15]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.Resize(63),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 5
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [17]:
def trainNetwork(network, optimizer, lossFunction, epochs, writer, device, name):
    network.train()
    for epoch in range(epochs):
        
        ### TRAINING ###
        trainLoss = 0
        correctTrain = 0
        totalTrain = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()
            
            # Calculate training accuracy
            _, predictions = torch.max(predictions, 1) 
            correctTrain += (predictions == labels).sum().item() 
            totalTrain += len(images)

            print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(trainLoader)}]', end='\r')
        

        trainLoss /= len(trainLoader)
        trainAccuracy = 100 * correctTrain / totalTrain
        writer.add_scalar(f"Loss/train - {name}", trainLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {name}", trainAccuracy, epoch)

        ### VALIDATION ###
        validLoss = 0
        correctValid = 0
        totalValid = 0
        for batch_nr, (images, labels) in enumerate(validLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

            # Calculate vaildation accuracy
            _, predictions = torch.max(predictions, 1) 
            correctValid += (predictions == labels).sum().item() 
            totalValid += len(images)

            print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(validLoader)}]', end='\r')

        validLoss /= len(validLoader)
        validAccuracy = 100 * correctValid / totalValid
        writer.add_scalar(f"Loss/valid - {name}", validLoss, epoch)
        writer.add_scalar(f"Accuracy/valid - {name}", validAccuracy, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss} \t Traning Acc: {trainAccuracy}% \t Validation Acc: {validAccuracy}%')

    writer.flush()

In [18]:
def testNetwork(network, device):
    network.eval()
    correctTest = 0
    totalTest = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  
            
            # Get predictions and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correctTest += (predictions == labels).sum().item() 
            totalTest += len(images)
            
            print(f'Batch [{batch_nr}/{len(testLoader)}]', end='\r')
        
    print(f"Test Accuracy: {100 * correctTest / totalTest}%")

In [19]:
class myAlexNet(nn.Module):
    def __init__(self, pretrained):
        super(myAlexNet,self).__init__()
        self.alexNet = torchvision.models.alexnet(pretrained=pretrained)
        self.out = nn.Linear(1000,10)
    
    def forward(self, x):
        x = F.relu(self.alexNet.forward(x))
        return self.out(x)


In [20]:
learningRate = 0.01
epochs = 1

In [21]:
networkNotTrained = myAlexNet(pretrained=False).to(device)

In [22]:
optimizerAdam = torch.optim.Adam(networkNotTrained.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss().to(device)

In [23]:
networkNotTrained.eval()

myAlexNet(
  (alexNet): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
     

In [24]:
# trainNetwork(networkNotTrained, optimizerAdam, lossFunction, epochs, writer, device, "Not pretrained")

In [25]:
networkPretrained = myAlexNet(pretrained=True).to(device)

In [26]:
for param in networkPretrained.alexNet.parameters():
    param.requires_grad=False

In [27]:
trainNetwork(networkPretrained, optimizerAdam, lossFunction, epochs, writer, device, "Pretrained weights")

KeyboardInterrupt: 

In [ ]:
testNetwork(networkPretrained)

Test Accuracy: 12.02%
